<a href="https://colab.research.google.com/github/naokityokoyama/Projetos_Diversos/blob/master/Caso_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O termo Produtividade mede a eficiência com que uma econômia usa seus recursos para produir bens e serviços. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
from fbprophet import Prophet

In [143]:
df= pd.read_csv('https://raw.githubusercontent.com/naokityokoyama/Projeto_extra/main/TFP.csv')

In [144]:
USA = df[df['isocode']=='USA']
MEX = df[df['isocode']=='MEX']
CAN = df[df['isocode']=='CAN']

In [13]:
df.head()

,isocode,year,rtfpna
0,USA,1950,0.617148
1,USA,1951,0.629588
2,USA,1952,0.638451
3,USA,1953,0.651858
4,USA,1954,0.646179


dataset com 183 instancias e com todas as features completas (183)

In [14]:
#confirmado nenhum dado nulo
df.isnull().sum()

isocode    0
year       0
rtfpna     0
dtype: int64

In [137]:
# rtfpna em relação ao tempo
trace1 = go.Scatter(x = USA['year'],
                    y = USA['rtfpna'],
                    mode = 'markers+lines',
                    name = 'USA')
# Gráfico de apenas linhas
trace2 = go.Scatter(x = CAN['year'],
                    y = CAN['rtfpna'],
                    mode = 'markers+lines',
                    name = 'CANADA')
# Criando gráfico com marcadores e linhas
trace3 = go.Scatter(x = MEX['year'],
                    y = MEX['rtfpna'],
                    mode = 'markers+lines',
                    name = 'MEXICO')

layout = go.Layout(title = 'Evolução Produtiva')
data = [trace1, trace2, trace3]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

O Mexico teve um aumento expressivo na evolução produtiva entre a década de 60 e final da decada de 70 e em seguinda um declínio bem considerado até inicio do ano 2010 que parece ter uma tendência a subir.

O Canada teve um pico de crescimento no meio da década de 50 e em seguinda uma leve queda, depois seguiu em crescimento até o ano 2000 onde se iniciou uma queda até os dias de hoje, parece está se estabilizando a essa queda no fim da serie. 

O EUA teve um crescimento a partir dos anos 50 em constante evolução (sem nenhum pico expressivo em toda a serie) tanto para queda quanto para subida.

In [146]:
df.groupby(['isocode']).size()

isocode
CAN    62
MEX    62
USA    62
dtype: int64

dataset com anos completos para os 3 paises

In [148]:
#pequeno resumo de estatistica descritiva dos dados
df.groupby(['isocode']).describe().T

isocode               CAN          MEX          USA
year   count    62.000000    62.000000    62.000000
       mean   1980.500000  1980.500000  1980.500000
       std      18.041619    18.041619    18.041619
       min    1950.000000  1950.000000  1950.000000
       25%    1965.250000  1965.250000  1965.250000
       50%    1980.500000  1980.500000  1980.500000
       75%    1995.750000  1995.750000  1995.750000
       max    2011.000000  2011.000000  2011.000000
rtfpna count    62.000000    62.000000    62.000000
       mean      0.992886     1.130132     0.803824
       std       0.050126     0.158209     0.116121
       min       0.843335     0.798159     0.617148
       25%       0.973294     1.012231     0.739949
       50%       1.004211     1.093859     0.766047
       75%       1.029513     1.299591     0.880544
       max       1.066104     1.383718     1.020392

Os dataset foram separados por paises

Modelando com Prophet 

In [100]:
dataset_usa = USA.drop('isocode', axis=1)
dataset_can = CAN.drop('isocode', axis=1)
dataset_mex = MEX.drop('isocode', axis=1)

In [101]:
dataset_usa.rename(columns={'year':'ds', 'rtfpna':'y'}, inplace=True)
dataset_can.rename(columns={'year':'ds', 'rtfpna':'y'}, inplace=True)
dataset_mex.rename(columns={'year':'ds', 'rtfpna':'y'}, inplace=True)

In [ ]:
#dataset_usa['DS'] = pd.to_datetime(dataset_usa['DS'],  format='%Y')

In [167]:
df.rename(columns={'DS':'ds', 'Y':'y'}, inplace=True)

In [109]:
dataset_usa['ds'] = pd.to_datetime(dataset_usa['ds'], format='%Y')
dataset_can['ds'] = pd.to_datetime(dataset_can['ds'], format='%Y')
dataset_mex['ds'] = pd.to_datetime(dataset_mex['ds'], format='%Y')

In [110]:
#criada uma função pois o Prophet espera sempre um df 
def prophet(dataset):
  m = Prophet()
  df = dataset
  return m.fit (df)

In [111]:
usa = prophet(dataset_usa)
can = prophet(dataset_can)
mex = prophet(dataset_mex)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [113]:
future_usa = usa.make_future_dataframe(periods=10, freq='Y')
future_can = can.make_future_dataframe(periods=10, freq='Y')
future_mex = mex.make_future_dataframe(periods=10, freq='Y')

In [138]:
forecast_usa = usa.predict(future_usa)
forecast_usa[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

forecast_can = can.predict(future_can)
forecast_can[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

forecast_mex = mex.predict(future_mex)
forecast_mex[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

forecast_can.head(2)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1950-01-01,0.386272,0.847824,0.912538,0.386272,0.386272,0.493607,0.493607,0.493607,0.493607,0.493607,0.493607,0.0,0.0,0.0,0.87988
1,1951-01-01,0.393592,0.854552,0.920384,0.393592,0.393592,0.493768,0.493768,0.493768,0.493768,0.493768,0.493768,0.0,0.0,0.0,0.88736


In [136]:
# Forecast 
trace1 = go.Scatter(x = forecast_usa['ds'],
                    y = forecast_usa['yhat'],
                    mode = 'markers+lines',
                    name = 'USA')
# Gráfico de apenas linhas
trace2 = go.Scatter(x = forecast_can['ds'],
                    y = forecast_can['yhat'],
                    mode = 'markers+lines',
                    name = 'CANADA')
# Criando gráfico com marcadores e linhas
trace3 = go.Scatter(x = forecast_mex['ds'],
                    y = forecast_mex['yhat'],
                    mode = 'markers+lines',
                    name = 'MEXICO')

layout = go.Layout(title = 'Forecast')
data = [trace1, trace2, trace3]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


Essa é a previsão para os proximos 10 anos

In [141]:
# Trend
trace1 = go.Scatter(x = forecast_usa['ds'],
                    y = forecast_usa['trend'],
                    mode = 'markers+lines',
                    name = 'USA')
# Gráfico de apenas linhas
trace2 = go.Scatter(x = forecast_can['ds'],
                    y = forecast_can['trend'],
                    mode = 'markers+lines',
                    name = 'CANADA')
# Criando gráfico com marcadores e linhas
trace3 = go.Scatter(x = forecast_mex['ds'],
                    y = forecast_mex['trend'],
                    mode = 'markers+lines',
                    name = 'MEXICO')

layout = go.Layout(title = 'Trend')
data = [trace1, trace2, trace3]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


Tendencia para os proximos anos temos uma estimativa da seguinte:
USA continuar a subir 
MEX tendendo a cair
CAN tendendo a equilibrio

In [142]:
# seasonality
trace1 = go.Scatter(x = forecast_usa['ds'],
                    y = forecast_usa['yearly'],
                    mode = 'markers+lines',
                    name = 'USA')
# Gráfico de apenas linhas
trace2 = go.Scatter(x = forecast_can['ds'],
                    y = forecast_can['yearly'],
                    mode = 'markers+lines',
                    name = 'CANADA')
# Criando gráfico com marcadores e linhas
trace3 = go.Scatter(x = forecast_mex['ds'],
                    y = forecast_mex['yearly'],
                    mode = 'markers+lines',
                    name = 'MEXICO')

layout = go.Layout(title = 'seasonality')
data = [trace1, trace2, trace3]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Depois de analisar o dataset as features que existem também para incrementar o modelo podemos destacar algumas

*    População (total da população) variavel **pop**
*    Emprego (Porcentagem de pessoas que trabalham) variavel **emp**
*    PIB (Produto interno Bruto) que podemos entender pela variavel **labsh ** pois seria o PIB por participação na força de trabalho

Com o numero da polulação vs Empregabilidade podemos ver a capacidade total de individuos para contribuir na produtividade
Com o PIB por produção podemos ver o desempenho da força de trabalho para elevar o PIB 
